<a href="https://colab.research.google.com/github/Aydin-ab/CV_DMTet/blob/main/Discriminator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# imports
from math import pi
import numpy as np
import igl
import torch

ModuleNotFoundError: No module named 'igl'

In [ ]:
# refinement, subdivision, refinement --> Discriminator (triangular mesh)

In [ ]:
# hyperparameters
N = 18 
r = 128
Kg_min = pi / 16
n_sample_patches = 10
iterations = 20000 

In [ ]:
# input
nvertices = 0 
n = nvertices
# vpred, fpred = igl.read_triangle_mesh(os.path.join(root_folder, "data", "bunny.off"))
# vgt, vgt = igl.read_triangle_mesh(os.path.join(root_folder, "data", "bunny.off"))
F = 100 #leave for now
vpred = np.ones((n, 3))
fpred = np.ones((F, 3))

In [ ]:
# shape of SDFs
Sreal = np.ones((N,N,N)) #mesh 
Sgt = np.ones((N,N,N))

In [ ]:
Kg = igl.gaussian_curvature(vgt, fgt)
Vgt = vgt.apply(Kg >= Kg_min )

In [ ]:
Sreal = SDF(Vgt + (Vgt.shape[0] - N/2) / r, (vgt, fgt))

In [ ]:
Sreal = SDF(Vgt + (Vgt.shape[0] - N/2) / r, (vgt, fgt))

In [ ]:
DISCRIMINATOR_CONFIG = {
#     'input_dim' : 3, # Coordinates of the grid's vertices
    'kernel_size' : [4, 3, 3, 3, 3]
    'out_channel' : [32, 64, 128, 256, 512],
    'stride' : [1, 2, 1, 1, 1]
    'output_dim' : 832, # SDF of the vertex input. The other "output" f_v comes from the prior activation layer of dimension 64
}

In [ ]:
# https://github.com/czq142857/DECOR-GAN/blob/3d736bf0f5bd9206cc26ee5336c1d7b4172f6cf8/evalFID.py#L63
class classifier(nn.Module):
    def __init__(self, config):
        super(classifier, self).__init__()
        self.config = config
        self.voxel_size = voxel_size

        self.conv_1 = nn.Conv3d(in_channels=1, out_channels=32, kernel_size=4, stride=1, bias=True)
        self.bn_1 = nn.InstanceNorm3d(32)

        self.conv_2 = nn.Conv3d(in_channels=32, out_channels=64, kernel_size=3, stride=2, bias=True)
        self.bn_2 = nn.InstanceNorm3d(64)

        self.conv_3 = nn.Conv3d(in_channels=64, out_channels=128, kernel_size=3, stride=1, bias=True)
        self.bn_3 = nn.InstanceNorm3d(128)

        self.conv_4 = nn.Conv3d(in_channels=128, out_channels=256, kernel_size=3, stride=1, bias=True)
        self.bn_4 = nn.InstanceNorm3d(256)

        self.conv_5 = nn.Conv3d(in_channels=256, out_channels=512, kernel_size=3, stride=1, bias=True)

#         if self.voxel_size==256:
#             self.bn_5 = nn.InstanceNorm3d(self.z_dim)
#             self.conv_5_2 = nn.Conv3d(self.z_dim, self.z_dim, 4, stride=2, padding=1, bias=True)

        self.linear1 = nn.Linear(512, 1, bias=True)



    def forward(self, inputs, is_training=False):
        out = inputs

        out = self.bn_1(self.conv_1(out))
        out = F.leaky_relu(out, negative_slope=0.01, inplace=True)

        out = self.bn_2(self.conv_2(out))
        out = F.leaky_relu(out, negative_slope=0.01, inplace=True)

        out = self.bn_3(self.conv_3(out))
        out = F.leaky_relu(out, negative_slope=0.01, inplace=True)

        out = self.bn_4(self.conv_4(out))
        out = F.leaky_relu(out, negative_slope=0.01, inplace=True)

        out = self.conv_5(out)

#         if self.voxel_size==256:
#             out = self.bn_5(out)
#             out = F.leaky_relu(out, negative_slope=0.01, inplace=True)
#             out = self.conv_5_2(out)

        z = F.adaptive_avg_pool3d(out, output_size=(1, 1, 1))
        z = z.view(-1,512)
        out = F.leaky_relu(z, negative_slope=0.01, inplace=True)
        
        # Add Fvol here (512) !!!
        
        out = self.linear1(out)

        return out, z